In [18]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil

import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
print(gpu.name)

Tesla P100-PCIE-16GB


In [19]:
import os
if not os.path.isdir('./utils'):
  !git clone https://github.com/lore-lml/machine-learning2020-hw3.git
  !mv 'machine-learning2020-hw3' 'utils'
  !rm './utils/hw3.ipynb'
  !rm './utils/README.md'

import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
from sklearn.model_selection import ParameterGrid
from utils.dann import alexnet
from utils.dann import train_no_dann, test_target, dann_train_src_target
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from utils.pacs_dataset import Pacs
from PIL import Image
from tqdm import tqdm
%matplotlib inline

Cloning into 'machine-learning2020-hw3'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 10087 (delta 37), reused 39 (delta 19), pack-reused 10029
Receiving objects: 100% (10087/10087), 175.15 MiB | 34.99 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Checking out files: 100% (9997/9997), done.


**Set Arguments**

In [20]:
DEVICE = 'cuda'

NUM_CLASSES = 7

BATCH_SIZE = 128      # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                      # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3             # The initial Learning Rate
MOMENTUM = 0.9        # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5   

NUM_EPOCHS = 1       # Total number of training epochs (iterations over dataset)
STEP_SIZE = 3         # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.5           # Multiplicative factor for learning rate step-down

ALPHA = 'dynamic'

param_grid = {
    "batch_size": [128],
    "epochs": [10],
    "lr": [5e-3, 8e-3],
    "gamma": [0.5],
    "step_size": [5],
    "alpha": [0.1]
}

BASE_FILE_PATH = "RUN_3"

**Define Data Preprocessing**

In [4]:
transforms = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [5]:
ROOT = 'utils/PACS'

photo = Pacs(ROOT, transform=transforms, source='photo')
cartoon = Pacs(ROOT, transform=transforms, source='cartoon')
sketch = Pacs(ROOT, transform=transforms, source='sketch')
art = Pacs(ROOT, transform=transforms, source='art_painting')

_, photo_labels = photo.get_img_with_labels()
_, cartoon_labels = cartoon.get_img_with_labels()
_, sketch_labels = sketch.get_img_with_labels()
_, art_labels = art.get_img_with_labels()

print(f"# classes source_data: {len(set(photo_labels))}")
print(f"# classes cartoon_data: {len(set(cartoon_labels))}")
print(f"# classes sketch_data: {len(set(sketch_labels))}")
print(f"# classes target_data: {len(set(art_labels))}")
print(f"source_data: {len(photo)} elements")
print(f"cartoon_data: {len(cartoon)} elements")
print(f"sketch_data: {len(sketch)} elements")
print(f"target_data: {len(art)} elements")
print(f"total images: {len(photo) + len(cartoon) + len(sketch) + len(art)}")

# classes source_data: 7
# classes cartoon_data: 7
# classes sketch_data: 7
# classes target_data: 7
source_data: 1670 elements
cartoon_data: 2344 elements
sketch_data: 3929 elements
target_data: 2048 elements
total images: 9991


**Prepare Dataloaders**

In [6]:
def get_train_loaders(dataset, batch_size=BATCH_SIZE):
  return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)

def get_eval_loaders(dataset, batch_size=BATCH_SIZE):
  return  DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)


**Prepare Network**

In [7]:
def get_alexnet(pretrained=False):
    return alexnet(pretrained)

def loss_optmizer_scheduler(model, lr=LR, step_size=STEP_SIZE, gamma=GAMMA):
  
  # Define loss function
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
  parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet
  
  optimizer = optim.SGD(parameters_to_optimize, lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  #optimizer = optim.Adam(parameters_to_optimize, lr=lr,amsgrad=True)
  #optimizer = optim.AdamW(parameters_to_optimize, lr=lr,amsgrad=True, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

  return criterion, optimizer, scheduler

**Training**

In [8]:
def saveResults(configuration, best_mean_accuracy, accuracies_test, class_losses, domain_losses=None,
                dann=False, append=False, base_path=BASE_FILE_PATH):
  suffix_path = "_results.txt"
  mode = "a" if append else "w"
  with open(f"{base_path}{suffix_path}", mode, encoding='utf-8') as f:
      title = "********** WITHOUD DANN **********" if not dann else "********** WITH DANN **********"
      f.write(f"{title}\n")
      f.write(f"Best Configuration:\n{configuration}\n\n")
      f.write(f"Best Mean Accuracy: {best_mean_accuracy}\n")
      f.write(f"Accuracies on Target:\n")
      join = ",".join(list(map(lambda x: str(x), accuracies_test)))
      f.write(f"{join}\n")
      f.write(f"Best Accuracy on Target: {max(accuracies_test)}\n")
      f.write("Class Losses:\n")
      join = ",".join(list(map(lambda x: str(x), class_losses)))
      f.write(f"{join}\n")
      if dann and domain_losses is not None:
        f.write("Domain Losses:\n")
        join = ",".join(list(map(lambda x: str(x), domain_losses)))
        f.write(f"{join}\n")
      f.write("\n\n")

  print("********* FILE SAVED *********")

In [9]:
def train(net, src_dataloader, test_dataloader, criterion, optimizer, scheduler, max_epoch=NUM_EPOCHS, device=DEVICE):
    train_losses = []
    accuracies = []
    
    net = net.to(device)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(max_epoch):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, max_epoch, scheduler.get_last_lr()))
        cumulative_loss, current_step = train_no_dann(net, src_dataloader, optimizer, criterion, current_step, device)
        curr_loss = cumulative_loss / len(src_dataloader)
        train_losses.append(curr_loss)
        scheduler.step()

        curr_accuracy = test_target(net, test_dataloader, device) / float(len(test_dataloader)*BATCH_SIZE)
        accuracies.append(curr_accuracy)
        print(f"\tCurrent Accuracy: {curr_accuracy}")
        
    return train_losses, accuracies

In [10]:
grid = dict(param_grid)
del grid['alpha']

grid = ParameterGrid(grid)
results = []
for config in grid:
  photo_dataloader = get_train_loaders(photo, config['batch_size'])
  cartoon_eval_dataloader = get_eval_loaders(cartoon, config['batch_size'])
  sketch_eval_dataloader = get_eval_loaders(sketch, config['batch_size'])
  no_dann_cartoon = get_alexnet(pretrained=True)
  criterion_cartoon, optimizer_cartoon, scheduler_cartoon = loss_optmizer_scheduler(no_dann_cartoon, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])
  no_dann_sketch = get_alexnet(pretrained=True)
  criterion_sketch, optimizer_sketch, scheduler_sketch = loss_optmizer_scheduler(no_dann_sketch, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])

  cartoon_losses, cartoon_accuracies = train(no_dann_cartoon, photo_dataloader, cartoon_eval_dataloader, 
                                                           criterion_cartoon, optimizer_cartoon, scheduler_cartoon, max_epoch=config['epochs'])
  sketch_losses, sketch_accuracies = train(no_dann_sketch, photo_dataloader, sketch_eval_dataloader, 
                                                         criterion_sketch, optimizer_sketch, scheduler_sketch, max_epoch=config['epochs'])

  curr_result = {'params': config, 'cartoon_losses': cartoon_losses, 'sketch_losses': sketch_losses, 
                 'best_acc': np.mean([max(cartoon_accuracies), max(sketch_accuracies)])}
  results.append(curr_result)
  
best_conf = max(results, key=lambda x: x['best_acc'])
print(f"Best conf:\n{best_conf['params']}")
print(f"Highest accuracy mean: {best_conf['best_acc']}")


Starting epoch 1/10, LR = [0.005]
Step 0, Loss_train 2.289766788482666
Step 10, Loss_train 0.3044741451740265


100%|██████████| 19/19 [00:06<00:00,  2.95it/s]

	Current Accuracy: 0.2680921052631579
Starting epoch 2/10, LR = [0.005]


Step 20, Loss_train 0.09047217667102814


100%|██████████| 19/19 [00:06<00:00,  2.95it/s]

...

Step 110, Loss_train 0.004683405160903931


100%|██████████| 31/31 [00:09<00:00,  3.31it/s]


	Current Accuracy: 0.31149193548387094
Starting epoch 10/10, LR = [0.004]
Step 120, Loss_train 0.003962565213441849


100%|██████████| 31/31 [00:09<00:00,  3.25it/s]

	Current Accuracy: 0.31149193548387094
Best conf:
{'batch_size': 128, 'epochs': 10, 'gamma': 0.5, 'lr': 0.008, 'step_size': 5}
Highest accuracy mean: 0.3338683149405772


In [12]:
# params = ParameterGrid(param_grid)[0]
params = best_conf['params']

photo_dataloader = get_train_loaders(photo, batch_size=params['batch_size'])
test_dataloader = get_eval_loaders(art, batch_size=params['batch_size'])
no_dann = get_alexnet(pretrained=True)
criterion, optimizer, scheduler = loss_optmizer_scheduler(no_dann, lr=params['lr'], step_size=params['step_size'], gamma=params['gamma'])

losses_no_dann, accuracies_no_dann = train(no_dann, photo_dataloader, test_dataloader, criterion, optimizer, scheduler, max_epoch=params['epochs'])
print(f"Photo transfer to Art accuracy WITHOUT DANN: {max(accuracies_no_dann)}")

saveResults(params, best_conf['best_acc'], accuracies_no_dann, losses_no_dann)
# saveResults(params, "_", accuracies_no_dann, losses_no_dann)

Starting epoch 1/10, LR = [0.008]
Step 0, Loss_train 2.146091938018799
Step 10, Loss_train 0.2671962380409241


100%|██████████| 16/16 [00:04<00:00,  3.22it/s]

	Current Accuracy: 0.43701171875
Starting epoch 2/10, LR = [0.008]


Step 20, Loss_train 0.1514684110879898


100%|██████████| 16/16 [00:04<00:00,  3.24it/s]

...

100%|██████████| 16/16 [00:04<00:00,  3.26it/s]

	Current Accuracy: 0.5009765625
Starting epoch 10/10, LR = [0.004]


Step 120, Loss_train 0.0005147606134414673


100%|██████████| 16/16 [00:04<00:00,  3.31it/s]


	Current Accuracy: 0.50341796875
Photo transfer to Art accuracy WITHOUT DANN: 0.52294921875
********* FILE SAVED *********


**Training with DANN**

In [15]:
import math
def dann_train_test(net, src_dataloader, tgt_dataloader, test_dataloader, criterion,
                    optimizer, scheduler, max_epoch=NUM_EPOCHS, alpha=ALPHA, file_path=BASE_FILE_PATH, device=DEVICE):
    
    class_losses = []
    domain_losses = []
    accuracies = []
    tolerance = 0
    
    net = net.to(device)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(max_epoch):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, max_epoch, scheduler.get_last_lr()))
        class_loss, domain_loss, current_step = dann_train_src_target(net, src_dataloader, 
                                          tgt_dataloader, optimizer, criterion, current_step,
                                          epoch, max_epoch, alpha=alpha, device=device)
        scheduler.step()
        if math.isnan(class_loss) or math.isnan(domain_loss):
          tolerance += 1
          if tolerance >= 3:
            print("EARLY STOPPING")
            break
        class_losses.append(class_loss)
        domain_losses.append(domain_loss)

        accuracy = test_target(net, test_dataloader, device) / float(len(test_dataloader)*BATCH_SIZE)
        accuracies.append(accuracy)
        print(f"Accuracy on test set: {accuracy}")
        
    return class_losses, domain_losses, accuracies

In [16]:
grid = ParameterGrid(param_grid)
dann_results = []
for config in grid:
  photo_dataloader = get_train_loaders(photo, config['batch_size'])
  cartoon_dataloader = get_train_loaders(cartoon, config['batch_size'])
  cartoon_eval_dataloader = get_eval_loaders(cartoon, config['batch_size'])
  sketch_dataloader = get_train_loaders(sketch, config['batch_size'])
  sketch_test_dataloader = get_eval_loaders(sketch, config['batch_size'])
  
  dann_cartoon = get_alexnet(pretrained=True)
  cartoon_criterion, optimizer_cartoon, scheduler_cartoon = loss_optmizer_scheduler(dann_cartoon, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])
  dann_sketch = get_alexnet(pretrained=True)
  sketch_criterion, optimizer_sketch, scheduler_sketch = loss_optmizer_scheduler(dann_sketch, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])

  class_losses_cartoon, domain_losses_cartoon, accuracies_cartoon = dann_train_test(
      dann_cartoon, photo_dataloader, cartoon_dataloader, cartoon_eval_dataloader,
      cartoon_criterion, optimizer_cartoon, scheduler_cartoon, max_epoch=config['epochs'], alpha=config['alpha']
  )

  class_losses_sketch, domain_losses_sketch, accuracies_sketch = dann_train_test(
      dann_sketch, photo_dataloader, sketch_dataloader, sketch_test_dataloader,
      sketch_criterion, optimizer_sketch, scheduler_sketch, max_epoch=config['epochs'], alpha=config['alpha']
  )

  curr_result = {'params': config, 'cartoon_losses': (class_losses_cartoon, domain_losses_cartoon), 'sketch_losses': (class_losses_sketch, domain_losses_sketch),
                 'best_acc': np.mean([max(accuracies_cartoon), max(accuracies_sketch)])}
  dann_results.append(curr_result)
  
best_conf = max(dann_results, key=lambda x: x['best_acc'])
print(f"Best configuration is:\n{best_conf['params']}")
print(f"Highest mean accuracy: {best_conf['best_acc']}")


Starting epoch 1/10, LR = [0.005]
Step 0
Class Loss 2.205806255340576, Domain Loss 1.4957263469696045
Step 10
Class Loss 0.158843994140625, Domain Loss 0.2693244516849518


100%|██████████| 19/19 [00:05<00:00,  3.59it/s]


Accuracy on test set: 0.24342105263157895
Starting epoch 2/10, LR = [0.005]
Step 20
Class Loss 0.14871039986610413, Domain Loss 0.37165355682373047


100%|██████████| 19/19 [00:05<00:00,  3.58it/s]


Accuracy on test set: 0.4309210526315789
Starting epoch 3/10, LR = [0.005]
Step 30
Class Loss 0.12302426248788834, Domain Loss 0.11611158400774002


...

Accuracy on test set: 0.19455645161290322
Starting epoch 5/10, LR = [0.008]


Step 60
Class Loss nan, Domain Loss nan


100%|██████████| 31/31 [00:08<00:00,  3.70it/s]

Accuracy on test set: 0.19455645161290322
Starting epoch 6/10, LR = [0.004]


Step 70
Class Loss nan, Domain Loss nan
EARLY STOPPING
Best configuration is:
{'alpha': 0.1, 'batch_size': 128, 'epochs': 10, 'gamma': 0.5, 'lr': 0.005, 'step_size': 5}
Highest mean accuracy: 0.40203867784380304


In [17]:
params = best_conf['params']
# params = ParameterGrid(param_grid)[0]

photo_dataloader = get_train_loaders(photo, params['batch_size'])
target_dataloader = get_train_loaders(art, params['batch_size'])
test_dataloader = get_eval_loaders(art, params['batch_size'])

dann = get_alexnet(pretrained=True)
class_criterion, optimizer, scheduler = loss_optmizer_scheduler(dann, lr=params['lr'], step_size=params['step_size'],
                                                                           gamma=params['gamma'])
class_losses, domain_losses, accuracies_dann = dann_train_test(dann, photo_dataloader, target_dataloader,
                          test_dataloader, class_criterion, optimizer, scheduler, max_epoch=params['epochs'], alpha=params['alpha'])

print(f"Photo transfer to Art accuracy DANN: {max(accuracies_dann)}")
saveResults(params, best_conf['best_acc'], accuracies_dann, class_losses, domain_losses=domain_losses,
            dann=True, append=True)
"""saveResults(params, "_", accuracies_dann, class_losses, domain_losses=domain_losses,
            dann=True, append=True)"""

Starting epoch 1/10, LR = [0.005]
Step 0
Class Loss 2.489631414413452, Domain Loss 1.8300917148590088
Step 10
Class Loss 0.13670220971107483, Domain Loss 0.5993434190750122


100%|██████████| 16/16 [00:04<00:00,  3.43it/s]


Accuracy on test set: 0.455078125
Starting epoch 2/10, LR = [0.005]
Step 20
Class Loss 0.0661679208278656, Domain Loss 0.4451800286769867


100%|██████████| 16/16 [00:04<00:00,  3.35it/s]


Accuracy on test set: 0.46728515625
Starting epoch 3/10, LR = [0.005]
Step 30
Class Loss 0.035510867834091187, Domain Loss 0.5779753923416138


...

100%|██████████| 16/16 [00:04<00:00,  3.39it/s]


Accuracy on test set: 0.5
Starting epoch 10/10, LR = [0.0025]
Step 120
Class Loss 0.001305554062128067, Domain Loss 0.10307952761650085


100%|██████████| 16/16 [00:04<00:00,  3.45it/s]

Accuracy on test set: 0.494140625
Photo transfer to Art accuracy DANN: 0.5
********* FILE SAVED *********


'saveResults(params, "_", accuracies_dann, class_losses, domain_losses=domain_losses,\n            dann=True, append=True)'